In [45]:
import os
import cv2
import sys
import json
import glob
import shutil
import codecs
import zipfile

In [46]:
os.getcwd()

'/home/rasho/Falling-Person-Detection-based-On-AlphaPose/dataset/DataPrepare'

In [47]:
path = '/home/rasho/Falling-Person-Detection-based-On-AlphaPose'
multicam = os.path.join(path,'input/multicam_dataset')

In [49]:
# Path where the videos are stored (in zip format, as in a fresh download)
output_base_path =  os.path.join(path,'input/multicam600/')
annotations_path = os.path.join(multicam,'annotations_multicam.json')
delays_file = os.path.join(multicam,'delays_multicam.json')
num_scenarios = 24
num_cameras = 8
W, H = 600, 600 # shape of new images (resize is applied)

# Extract all files and organise them (if necessary)
dir = glob.glob(multicam+'/*')

In [41]:
dir

['/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute12',
 '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/delays_multicam.json',
 '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute03',
 '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute15',
 '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute24',
 '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute07',
 '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute22',
 '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/annotations_multicam.json',
 '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute19',
 '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute23',
 '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/i

In [50]:
with open(annotations_path, 'r') as json_file:
    annotations = json.load(json_file)
    
with open(delays_file, 'r') as json_file:
    delays = json.load(json_file)

In [51]:
 annotations 

{'scenario1': {'reference': 11, 'start': [1080], 'end': [1108]},
 'scenario2': {'reference': 4, 'start': [375], 'end': [399]},
 'scenario3': {'reference': 11, 'start': [591], 'end': [625]},
 'scenario4': {'reference': 6, 'start': [288], 'end': [314]},
 'scenario5': {'reference': 11, 'start': [311], 'end': [336]},
 'scenario6': {'reference': 1, 'start': [583], 'end': [629]},
 'scenario7': {'reference': 6, 'start': [476], 'end': [507]},
 'scenario8': {'reference': 4, 'start': [271], 'end': [298]},
 'scenario9': {'reference': 11, 'start': [628], 'end': [651]},
 'scenario10': {'reference': 11, 'start': [512], 'end': [530]},
 'scenario11': {'reference': 7, 'start': [464], 'end': [489]},
 'scenario12': {'reference': 11, 'start': [605], 'end': [653]},
 'scenario13': {'reference': 4, 'start': [823], 'end': [863]},
 'scenario14': {'reference': 6, 'start': [989], 'end': [1023]},
 'scenario15': {'reference': 11, 'start': [755], 'end': [787]},
 'scenario16': {'reference': 4, 'start': [891], 'end':

In [44]:
# For each scenario
for s in range(1,num_scenarios+1):
    # Get all videos (one per camera)
    videos = glob.glob(multicam + '/chute{:02}/*'.format(s))
    print(videos)
    videos.sort()
    starts = annotations['scenario{}'.format(s)]['start']
    ends = annotations['scenario{}'.format(s)]['end']

    for cam, video in enumerate(videos, 1):
        cap = cv2.VideoCapture(video)
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        # Delay of this camara for this scenario
        delay = delays['camera{}'.format(cam)][str(s)]
        for start, end in zip(starts, ends):
            # Apply the delay
            start += delay
            end += delay
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

            # Read the pre-fall part
            pos = 0
            while pos < start:
                ret, frame = cap.read()
                pos += 1
                output_path = (
                    output_base_path +
                    'chute{:02}/NotFalls/camera{}_pre/'.format(
                    s, cam
                ))
                if not os.path.exists(output_path):
                    os.makedirs(output_path)
                cv2.imwrite(output_path + 'img_{:05d}.jpg'.format(int(pos)),
                        cv2.resize(frame, (W,H)),
                        [int(cv2.IMWRITE_JPEG_QUALITY), 95])
  
            # Read the fall part
            assert cap.get(cv2.CAP_PROP_POS_FRAMES) == start
            while pos <= end:
                ret, frame = cap.read()
                pos += 1
                output_path = (
                    output_base_path +
                    'chute{:02}/Falls/camera{}/'.format(
                    s, cam
                ))
                if not os.path.exists(output_path):
                    os.makedirs(output_path)
                cv2.imwrite(output_path + 'img_{:05d}.jpg'.format(int(pos)),
                        cv2.resize(frame, (W,H)),
                        [int(cv2.IMWRITE_JPEG_QUALITY), 95])
            # Read the post-fall part
            assert cap.get(cv2.CAP_PROP_POS_FRAMES) == end + 1
            while pos < length:
                ret, frame = cap.read()
                pos += 1
                output_path = (
                    output_base_path +
                    'chute{:02}/NotFalls/camera{}_post/'.format(
                    s, cam
                ))
                if not os.path.exists(output_path):
                    os.makedirs(output_path)
                cv2.imwrite(output_path + 'img_{:05d}.jpg'.format(int(pos)),
                        cv2.resize(frame, (W,H)),
                        [int(cv2.IMWRITE_JPEG_QUALITY), 95])
        # If there was no fall
        if len(starts) == 0 and len(ends) == 0:
            pos = 0
            while pos < length:
                ret, frame = cap.read()
                pos += 1
                output_path = (
                    output_base_path +
                    'chute{:02}/NotFalls/camera{}_full/'.format(
                    s, cam
                ))
                if not os.path.exists(output_path):
                    os.makedirs(output_path)
                cv2.imwrite(output_path + 'img_{:05d}.jpg'.format(int(pos)),
                        cv2.resize(frame, (W,H)),
                        [int(cv2.IMWRITE_JPEG_QUALITY), 95])

['/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute01/cam6.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute01/cam1.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute01/cam4.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute01/cam2.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute01/cam7.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute01/cam5.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute01/cam3.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute01/cam8.avi']
['/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute02/cam6.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute02/cam1.avi', '/home/r

['/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute12/cam6.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute12/cam1.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute12/cam4.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute12/cam2.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute12/cam7.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute12/cam5.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute12/cam3.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute12/cam8.avi']
['/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute13/cam6.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute13/cam1.avi', '/home/r

['/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute23/cam6.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute23/cam1.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute23/cam4.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute23/cam2.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute23/cam7.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute23/cam5.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute23/cam3.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute23/cam8.avi']
['/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute24/cam6.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute24/cam1.avi', '/home/r

In [54]:
# For each scenario
for s in range(1,num_scenarios+1):
    # Get all videos (one per camera)
    videos = glob.glob(multicam + '/chute{:02}/*'.format(s))
    print(videos)
    videos.sort()
    starts = annotations['scenario{}'.format(s)]['start']
    ends = annotations['scenario{}'.format(s)]['end']
    index = 0
    for cam, video in enumerate(videos, 1):
        cap = cv2.VideoCapture(video)
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        # Delay of this camara for this scenario
        delay = delays['camera{}'.format(cam)][str(s)]
        for start, end in zip(starts, ends):
            # Apply the delay
            start += delay
            end += delay
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

            # Read the pre-fall part
            pos = 0
            while pos < start:
                ret, frame = cap.read()
                pos += 1
                index +=1
                output_path = (output_base_path +'/NotFalls/')
                if not os.path.exists(output_path):
                    os.makedirs(output_path)
                cv2.imwrite(output_path + 'img_{:05d}.jpg'.format(int(index)),
                        cv2.resize(frame, (W,H)),
                        [int(cv2.IMWRITE_JPEG_QUALITY), 95])
  
            # Read the fall part
            assert cap.get(cv2.CAP_PROP_POS_FRAMES) == start
            while pos <= end:
                ret, frame = cap.read()
                pos += 1
                index +=1
                output_path = (output_base_path+'/Falls/')
                if not os.path.exists(output_path):
                    os.makedirs(output_path)
                cv2.imwrite(output_path + 'img_{:05d}.jpg'.format(int(index)),
                        cv2.resize(frame, (W,H)),
                        [int(cv2.IMWRITE_JPEG_QUALITY), 95])
            # Read the post-fall part
            assert cap.get(cv2.CAP_PROP_POS_FRAMES) == end + 1
            while pos < length:
                ret, frame = cap.read()
                pos += 1
                index +=1
                output_path = (
                    output_base_path +'/NotFalls/')
                if not os.path.exists(output_path):
                    os.makedirs(output_path)
                cv2.imwrite(output_path + 'img_{:05d}.jpg'.format(int(index)),
                        cv2.resize(frame, (W,H)),
                        [int(cv2.IMWRITE_JPEG_QUALITY), 95])
        # If there was no fall
        if len(starts) == 0 and len(ends) == 0:
            pos = 0
            index +=1
            while pos < length:
                ret, frame = cap.read()
                pos += 1
                index +=1
                output_path = (output_base_path +'/NotFalls/')
                if not os.path.exists(output_path):
                    os.makedirs(output_path)
                cv2.imwrite(output_path + 'img_{:05d}.jpg'.format(int(index)),
                        cv2.resize(frame, (W,H)),
                        [int(cv2.IMWRITE_JPEG_QUALITY), 95])

['/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute01/cam6.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute01/cam1.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute01/cam4.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute01/cam2.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute01/cam7.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute01/cam5.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute01/cam3.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute01/cam8.avi']
['/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute02/cam6.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute02/cam1.avi', '/home/r

['/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute12/cam6.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute12/cam1.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute12/cam4.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute12/cam2.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute12/cam7.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute12/cam5.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute12/cam3.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute12/cam8.avi']
['/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute13/cam6.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute13/cam1.avi', '/home/r

['/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute23/cam6.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute23/cam1.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute23/cam4.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute23/cam2.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute23/cam7.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute23/cam5.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute23/cam3.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute23/cam8.avi']
['/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute24/cam6.avi', '/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/multicam_dataset/chute24/cam1.avi', '/home/r